<a href="https://colab.research.google.com/github/MHoffmannAC/wordle/blob/main/Wordle_by_MHoffmann_and_ChatGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
# @title Main code definitions (colors, languages and wordle-class)

from IPython.display import display, HTML, clear_output, Javascript
import ipywidgets as widgets
import urllib.request
import random
from time import sleep

# Define color schemes for different states of the game
colors = {
    'empty': {'fg': "silver", 'bg': "white"},        # Empty cells on the board
    'correct': {'fg': "white", 'bg': "green"},       # Correct letter in the correct position
    'wrong': {'fg': "white", 'bg': "gray"},          # Incorrect letter
    'in': {'fg': "white", 'bg': "orange"},           # Correct letter in the wrong position
    'alphabet': {'fg': "black", 'bg': "lightgray"}   # Default color for the alphabet display
}

# Define messages for different languages
messages = {
    'en': {
        'congratulations': "Congratulations! You found the word!",
        'try_again': "Not a valid word or incorrect length. Try again.",
        'sorry': "Sorry, you didn't find the word. The solution was {solution}.",
        'enter_valid_word': "Please enter a valid 5-letter word.",
        'random_word': "Use Random Word",
        'custom_word': "Enter Your Own Word",
        'guess_placeholder': 'Enter a 5-letter word',
        'guess_label': 'Your Guess:',
        'solution_label': 'Your Solution:',
        'submit_guess': 'Submit Guess',
        'restart_game': 'Restart Game',
        'welcome_header': 'Welcome to WORDLE',
        'select_word': 'Please choose what kind of word you would like:',
        'select_userword': 'Please choose a 5-letter-word as solution:',
        'select_language': 'or choose a different language:'
    },
    'de': {
        'congratulations': "Herzlichen Glückwunsch! Du hast das Wort gefunden!",
        'try_again': "Kein gültiges Wort oder falsche Länge. Versuche es erneut.",
        'sorry': "Entschuldigung, du hast das Wort nicht gefunden. Die Lösung war {solution}.",
        'enter_valid_word': "Bitte gib ein gültiges 5-Buchstaben-Wort ein.",
        'random_word': "Zufälliges Wort",
        'custom_word': "Eigenes Wort",
        'guess_placeholder': 'Gib ein 5-Buchstaben-Wort ein',
        'guess_label': 'Dein Wort:',
        'solution_label': 'Dein Lösungswort:',
        'submit_guess': 'Wort absenden',
        'restart_game': 'Spiel neu starten',
        'welcome_header': 'Willkommen zu WORDLE',
        'select_word': 'Wähle, welche Art von Wort du gerne hättest:',
        'select_userword': 'Wähle ein 5-buchstabiges Lösungswort:',
        'select_language': 'oder ändere die Sprache:'
    }
    # Add more languages here if needed
}

class Wordle:
    def __init__(self, language='en', max_guesses=6):
        """
        Initializes the Wordle game with default settings.
        """
        self.max_guesses = max_guesses       # Maximum number of allowed guesses
        self.num_guess = 0                   # Current number of guesses made
        self.found_solution = False          # Flag to indicate if the solution has been found
        self.guessed_letters = set()         # Set to store all guessed letters
        self.words = []                      # List to store valid words fetched from the URL
        self.solution = ""                   # The word to be guessed
        self.alphabet_colors = {letter: colors['alphabet']['bg'] for letter in "ABCDEFGHIJKLMNOPQRSTUVWXYZ"}
        self.set_language(language)          # Set language (default: English) and get word list
        self.show_settings()                 # Show settings screen

    def show_settings(self):
        """
        Settings Screen: Displays buttons for word or language selection.
        """
        clear_output(wait=True)  # Clear previous outputs for a clean interface

        # Add a header and description for language selection
        header = f"<h2>{messages[self.language]['welcome_header']}</h2>"
        description_word = f"<p>{messages[self.language]['select_word']}</p>"
        description_lang = f"<p>{messages[self.language]['select_language']}</p>"

        # Create buttons for English and German languages
        lang_button_en =      widgets.Button(description="English")
        lang_button_de =      widgets.Button(description="Deutsch")
        word_button_random =  widgets.Button(description=messages[self.language]['random_word'])
        word_button_user =    widgets.Button(description=messages[self.language]['custom_word'])

        # Bind button clicks to the select_language method with appropriate language codes
        lang_button_en.on_click(lambda btn: self.__init__(language='en'))
        lang_button_de.on_click(lambda btn: self.__init__(language='de'))
        word_button_random.on_click(self.select_random)
        word_button_user.on_click(self.get_user_solution)

        # Display the header, description, and language selection buttons
        display(HTML(header))
        display(HTML(description_word))
        display(word_button_random)
        display(word_button_user)
        display(HTML(description_lang))
        display(lang_button_en)
        display(lang_button_de)


    def set_language(self, language_code):
        """
        Sets the word list URL based on the selected language and proceeds to word selection.

        Parameters:
            language_code (str): The code of the selected language (e.g., 'en' or 'de').
        """

        # Set the URL for the word list based on the selected language
        if language_code == 'en':
            self.url = "https://gist.githubusercontent.com/dracos/dd0668f281e685bad51479e5acaadb93/raw/6bfa15d263d6d5b63840a8e5b64e04b382fdb079/valid-wordle-words.txt"
            self.language = 'en'
        elif language_code == 'de':
            self.url = "https://codeberg.org/davidak/wortliste/raw/branch/master/wortliste.txt"
            self.language = 'de'
        # Add more languages here as needed. E.g.:
        # elif language_code == 'fr':
        #     self.url = "https://example.com/french-words.txt"
        #     self.language = 'fr'

    def fetch_words_from_url(self):
        """
        Fetches words from the specified URL and filters them to include only 5-letter words.
        """
        response = urllib.request.urlopen(self.url)  # Open the URL
        # Decode each line, strip whitespace, convert to uppercase, and filter for 5-letter words
        self.words = [line.decode('utf-8').strip().upper() for line in response if len(line.decode('utf-8').strip()) == 5]

    def select_random(self, button):
        """
        Selects a random word from the fetched word list as the solution and starts the game.

        Parameters:
            button (ipywidgets.Button): The button that was clicked (unused but mandatory).
        """
        self.fetch_words_from_url()               # Fetch the word list
        self.solution = random.choice(self.words)  # Randomly select the solution word
        self.start_game()                          # Begin the game

    def get_user_solution(self, button):
        """
        Allows the user to input their own solution word and validates it.

        Parameters:
            button (ipywidgets.Button): The button that was clicked (unused but mandatory).
        """
        self.fetch_words_from_url()  # Fetch the word list before allowing user input
        clear_output(wait=True)  # Clear previous outputs for a clean interface

        header = f"<h2>{messages[self.language]['welcome_header']}</h2>"
        description_select = f"<p>{messages[self.language]['select_userword']}</p>"

        # Create a password input box for the user to enter their own word (hidden input)
        input_box = widgets.Password(
            placeholder=messages[self.language]['guess_placeholder'],
            description=messages[self.language]['solution_label'],
            disabled=False,
            style={'description_width': 'initial'}
        )
        # Create a submit button for the user's word
        submit_word_button = widgets.Button(description=messages[self.language]['submit_guess'])
        # Bind the submit button to the set_user_solution method
        submit_word_button.on_click(lambda btn: self.set_user_solution(input_box.value.upper()))
        # Also bind the Enter key in the input box to submit the word
        input_box.on_submit(lambda btn: self.set_user_solution(input_box.value.upper()))

        input_box.add_class('custom-password-widget')


        # Display the input box and submit button
        display(HTML(header))
        display(HTML(description_select))
        display(input_box)
        display(submit_word_button)

    def set_user_solution(self, user_word):
        """
        Sets the user's input as the solution if it is valid and starts the game.

        Parameters:
            button (ipywidgets.Button): The button that was clicked (unused but mandatory).
        """
        # Check if the word is valid (exists in the word list and has 5 letters)
        if user_word in self.words and len(user_word) == 5:
            self.solution = user_word  # Set the user's word as the solution
            self.start_game()           # Begin the game
        else:
            # Display an error message if the input is invalid
            display(HTML(f"<p style='color:red;'>{messages[self.language]['enter_valid_word']}</p>"))

    def start_game(self):
        """
        Initializes the game state and sets up the game interface.
        """
        self.num_guess = 0  # Reset the number of guesses
        self.found_solution = False  # Reset the solution found flag
        # Create an empty game board for displaying guesses
        self.game_board = [[
            {'letter': '', 'fg': colors['empty']['fg'], 'bg': colors['empty']['bg']} for _ in range(len(self.solution))
        ] for _ in range(self.max_guesses)]

        # Create an input box for guesses
        self.input_box = widgets.Text(
            placeholder=messages[self.language]['guess_placeholder'],
            description=messages[self.language]['guess_label'],
            disabled=False,
            style={'description_width': 'initial'}
        )
        # Create a submit button for guesses
        self.submit_button = widgets.Button(description=messages[self.language]['submit_guess'])
        # Create a restart button
        self.restart_button = widgets.Button(description=messages[self.language]['restart_game'])

        # Bind actions to buttons
        self.submit_button.on_click(self.handle_input)
        self.restart_button.on_click(self.restart_game)
        self.input_box.on_submit(self.handle_input_from_submit)

        # Display the game interface
        self.update_display()

    def handle_input_from_submit(self, text):
        """
        Handles the Enter key press in the input box by calling the handle_input method.

        Parameters:
            text (str): The text entered in the input box (unused but mandatory).
        """
        self.handle_input(None)  # Pass None since the button parameter is not used

    def handle_input(self, button):
        """
        Processes the player's guess, updates the game state, and provides feedback.

        Parameters:
            button (ipywidgets.Button): The button that was clicked (unused but mandatory).
        """
        user_guess = self.input_box.value.strip().upper()  # Retrieve and clean the guess
        self.input_box.value = ''  # Clear the input box for the next guess

        # Validate the guess: correct length and exists in the word list
        if len(user_guess) == len(self.solution) and user_guess in self.words:
            self.update_board(user_guess)  # Update the game board with the guess
            self.num_guess += 1            # Increment the guess count

            if user_guess == self.solution:
                # Player has found the correct word
                self.found_solution = True
                self.update_display()  # Update the display to show the final state
                display(HTML(f"<h2>{messages[self.language]['congratulations']}</h2>"))
                self.submit_button.disabled = True  # Disable further guesses
            elif self.num_guess >= self.max_guesses:
                # Player has used all guesses without finding the word
                self.update_display()
                display(HTML(f"<h2>{messages[self.language]['sorry'].format(solution=self.solution)}</h2>"))
                self.submit_button.disabled = True  # Disable further guesses
            else:
                # Continue the game with the next guess
                self.update_display()
        else:
            # Display an error message for invalid guesses
            display(HTML(f"<p style='color:red;'>{messages[self.language]['try_again']}</p>"))

    def update_board(self, guess):
        """
        Updates the game board and alphabet colors based on the player's guess.

        Parameters:
            guess (str): The player's guessed word.
        """
        self.guessed_letters.update(set(guess))  # Add guessed letters to the set

        # Count the occurrences of each letter in the solution
        solution_letter_count = {letter: self.solution.count(letter) for letter in set(self.solution)}
        matched_positions = [False] * len(self.solution)  # Track correctly matched positions

        # First pass: Identify and mark correct letters (green)
        for i in range(len(guess)):
            if guess[i] == self.solution[i]:
                self.game_board[self.num_guess][i] = {
                    'letter': guess[i],
                    'fg': colors['correct']['fg'],
                    'bg': colors['correct']['bg']
                }
                matched_positions[i] = True
                solution_letter_count[guess[i]] -= 1  # Decrement the count for matched letters

                # Update the alphabet color to green
                self.alphabet_colors[guess[i]] = colors['correct']['bg']

        # Second pass: Identify and mark letters that are in the solution but misplaced (orange) or wrong (gray)
        for i in range(len(guess)):
            if not matched_positions[i]:  # Skip already correctly placed letters
                if guess[i] in self.solution and solution_letter_count[guess[i]] > 0:
                    self.game_board[self.num_guess][i] = {
                        'letter': guess[i],
                        'fg': colors['in']['fg'],
                        'bg': colors['in']['bg']
                    }
                    solution_letter_count[guess[i]] -= 1  # Decrement the count for misplaced letters

                    # Update the alphabet color to orange if it's not already green
                    if self.alphabet_colors[guess[i]] != colors['correct']['bg']:
                        self.alphabet_colors[guess[i]] = colors['in']['bg']
                else:
                    self.game_board[self.num_guess][i] = {
                        'letter': guess[i],
                        'fg': colors['wrong']['fg'],
                        'bg': colors['wrong']['bg']
                    }

                    # Update the alphabet color to gray if it's not already green or orange
                    if self.alphabet_colors[guess[i]] not in [colors['correct']['bg'], colors['in']['bg']]:
                        self.alphabet_colors[guess[i]] = colors['wrong']['bg']

    def print_board(self):
        """
        Generates the HTML representation of the game board with styled cells.

        Returns:
            str: HTML string representing the styled game board.
        """
        styled_text = ""
        for row in self.game_board:
            styled_text += "<div style=\"display: flex;\">\n"  # Start a new row
            for cell in row:
                # Create a styled div for each cell with appropriate background and foreground colors
                styled_text += f"<div style=\"background-color: {cell['bg']}; color: {cell['fg']}; padding: 10px; margin: 2px; font-family: monospace; font-size: 24px; border: 1px solid black; width: 20px; height: 25px; text-align: center;\">{cell['letter']}</div>\n"
            styled_text += "</div>\n"  # End the row
        return styled_text

    def print_alphabet(self):
        """
        Generates the HTML representation of the alphabet display with colored letters.

        Returns:
            str: HTML string representing the styled alphabet.
        """
        alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
        styled_text = "<div style=\"display: flex; flex-wrap: wrap; width: 250px;\">\n"

        # Iterate through each letter in the alphabet and style it based on its current color
        for letter in alphabet:
            bg_color = self.alphabet_colors[letter]  # Get the background color for the letter
            styled_text += f"<div style=\"background-color: {bg_color}; color: {colors['alphabet']['fg']}; padding: 5px; margin: 2px; font-family: monospace; font-size: 16px; border: 1px solid black; width: 20px; text-align: center;\">{letter}</div>\n"

        styled_text += "</div>\n"  # End the alphabet display
        return styled_text

    def update_display(self):
        """
        Clears the current display and renders the updated game board, alphabet, and input widgets.
        """
        clear_output(wait=True)            # Clear previous outputs
        display(HTML(self.print_board()))  # Display the game board
        display(HTML(self.print_alphabet()))  # Display the alphabet with colored letters
        display(self.input_box)            # Display the guess input box
        display(self.submit_button)        # Display the submit guess button
        display(self.restart_button)       # Display the restart game button

    def restart_game(self, button):
        """
        Restarts the game by reinitializing the Wordle instance.

        Parameters:
            button (ipywidgets.Button): The button that was clicked (unused but mandatory).
        """
        clear_output(wait=True)  # Clear the current display
        self.__init__(language=self.language)          # Reinitialize the game to start over



In [61]:
# Initialize the game instance
game = Wordle()

Button(description='Zufälliges Wort', style=ButtonStyle())

Button(description='Eigenes Wort', style=ButtonStyle())

Button(description='English', style=ButtonStyle())

Button(description='Deutsch', style=ButtonStyle())